In [ ]:
import numpy as np
import json
import os
from scipy.spatial import distance

from allennlp.common.file_utils import cached_path

from sklearn.utils import shuffle
from scipy.spatial import distance
import torch
from sklearn.metrics.cluster import v_measure_score

import sys

task = "bios"
split = "dev"

model_path = f"../experiments/models/{task}/roberta-large"

dev_encodings_normal = np.load(model_path + f"/encodings/predicted_{task}_{split}_encoded_representations.npy")
dev_preds_normal = np.load(model_path + f"/encodings/predicted_{task}_{split}_predictions.npy")
enc_size = dev_encodings_normal.shape[1]

w = np.load(model_path + "/w.npy")  # shape: (num_classes, enc_size)
b = np.load(model_path + "/b.npy")  # shape: (num_classes)
with open(model_path + "/label2index.json", "r") as f:
    label2index = json.load(f)
    index2label = {label2index[k]: k for k in label2index}
    
index2label

In [ ]:

dev_path = f"../data/{task}/{split}.jsonl"

with open(cached_path(dev_path)) as f:
    dev_data = [json.loads(line) for line in f if line.strip() if line.strip()]
# print([d['gold_label'] for d in dev_data])
dev_labels = np.array([label2index[d['label']] for d in dev_data])


In [ ]:
dev_data[0]

In [ ]:

import pandas as pd
from scipy.special import softmax
from scipy.stats import entropy

orig_encodings = dev_encodings_normal
orig_labels = dev_labels
orig_predictions = dev_preds_normal

classifier_w = w
classifier_b = b

def dkl(p,q):
    p = p / p.sum(axis=1).reshape(-1, 1)
    q = q / q.sum(axis=1).reshape(-1, 1)
    return entropy(p,q, axis=1) + entropy(q,p, axis=1)

for fact in index2label:
    drop = []
    print(f"\t\t\t\t\tfact: {index2label[fact]}")
    for foil in index2label:
        if fact == foil:
            drop.append(-1000)
            continue
            
        contrastive_examples = np.any([orig_predictions == fact], axis=0)
    
        encodings = orig_encodings[contrastive_examples]
        predictions = orig_predictions[contrastive_examples]
        
        if len(encodings) == 0:
            drop.append(0)
            continue

        u = classifier_w[fact] - classifier_w[foil]
        contrastive_rowspace_projection = np.outer(u, u) / u.dot(u)
    
        encodings_in_contrastive_rowspace = encodings @ contrastive_rowspace_projection #@ directional_nullspace

        assert (predictions == np.argmax(encodings @ classifier_w.T + classifier_b, axis=1)).all()

        predictions_in_contrastive_space = np.argmax(encodings_in_contrastive_rowspace @ classifier_w.T + classifier_b, axis=1)

        prediction_probabilities = softmax(encodings @ classifier_w.T + classifier_b, axis=1)
        prediction_probabilities_contrastive = softmax(encodings_in_contrastive_rowspace @ classifier_w.T + classifier_b, axis=1)

        dist10 = dkl(prediction_probabilities_contrastive,
                    prediction_probabilities).mean()

        drop.append(dist10)

    tab = []
    print(fact)
    for c in range(len(index2label)):
        if c == fact:
            continue
        tab.append({
            'foil idx': c,
            'foil': index2label[c],
            'Dkl': drop[c],
        })

    tab = sorted(tab, key=lambda v: -v['Dkl'])

    display(pd.DataFrame(tab))
    